In [1]:
import json

import pandas as pd
import plotly.express as px

In [2]:
data = pd.read_csv("data.csv")
data = data[~data["address"].isna()]

def data_query(*, min_beds, min_baths, min_cars):

    transformed_data = data.query(f"beds >= {min_beds} & baths >= {min_baths} & cars >= {min_cars}")
    transformed_data = transformed_data.groupby(by="suburb")["price"].agg(["median", "mean", "std", "count"])

    transformed_data.index = transformed_data.index.str.upper()
    transformed_data = transformed_data.reset_index()

    return transformed_data

In [3]:
with open("qld_suburb_geo.json") as j:
    geojson = json.load(j)

In [4]:
LONGITUDE_MAX = -27.15
LONGITUDE_MIN = -27.8
LATITUDE_MAX = 153.6
LATITUDE_MIN = 152.8

brisbane_idxs = []
for n, feature in enumerate(geojson["features"]):
    for lat, long in feature["geometry"]["coordinates"][0][0]:
        if long > LONGITUDE_MAX or long < LONGITUDE_MIN or lat > LATITUDE_MAX or lat < LATITUDE_MIN:
            break
    else:
        brisbane_idxs.append(n)

geojson["features"] = [f for n, f in enumerate(geojson["features"]) if n in brisbane_idxs]

In [5]:
for feature in geojson["features"]:
    feature["id"] = feature["properties"]["qld_loca_2"]

In [16]:
def plot_brisbane_house_prices(*, min_beds, min_baths, min_cars):
    data = data_query(min_beds=min_beds, min_baths=min_baths, min_cars=min_cars)

    # Slim geojson to just suburbs in plot to minimise file size
    suburbs_list = data["suburb"].unique().tolist()
    geojson["features"] = [
        f 
        for f in geojson["features"]
        if f["properties"]["qld_loca_2"] in suburbs_list
    ]


    fig = px.choropleth_mapbox(
        data, 
        geojson=geojson,
        locations="suburb", 
        color="median",
        color_continuous_scale="Viridis",
        range_color=(0, 3000000),
        mapbox_style="carto-positron",
        zoom=11, center = {"lat": -27.45, "lon": 153},
        opacity=0.5,
        custom_data=["count", "mean"]
    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, height=1300, width=1600)
    fig.update_traces(
        hovertemplate="<br>".join(
            [
                "suburb: %{location}",
                "median=%{z}",
                "houses sold=%{customdata[0]}",
                "mean=%{customdata[1]}",
            ]
        ) + "<extra></extra>"
    )
    fig.write_html("plot.html")
    return fig


In [17]:
plot_brisbane_house_prices(
    min_beds=3,
    min_baths=1,
    min_cars=0,
)
pass  # To prevent plot showing here and making ipynb file huge